In [1]:
cd ../..

/people/gerald/Documents/repositories/Educational-French-Question-Answering


## Translate corpus output
The objective of this notebook is to produce datasets with output (questions) in different english while context being in french and reversly. To this end we leverage translation model MBart on the questions of the originals datasets.

In [2]:
import json, os
from src.data_utils import corpus, pb_corpus
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast



In [3]:
os.environ['EFQADATA'] = '/people/gerald/Documents/repositories/Educational-French-Question-Answering/dataset'

In [4]:
data_folder = os.path.expandvars("$EFQADATA/source")
datasets_name = ["squad.pb.json","fquad.pb.json", "piaf.pb.json", "freelance.pb.json", "isahit.pb.json"]
datasets = {}

for dataset_name in datasets_name: 
    with open(os.path.join(data_folder, dataset_name)) as f:
        data = json.load(f)
        if('test' not in data):
            datasets[dataset_name.split('.')[0]] = {'test': data}
        else:
            datasets[dataset_name.split('.')[0]] = data


In [5]:
LANGUAGE_MAP = {
    "fr" : "fr_XX",
    "en" : "en_XX",
    "es" : "es_XX",
    "ja" : "ja_XX",
    "pt" : "pt_XX",
    "tl" : "tl_XX"
}

In [6]:
pretrained_name="facebook/mbart-large-50-many-to-many-mmt"
load_lang=["fr", "en"]
model = MBartForConditionalGeneration.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")
tokenizer = MBart50TokenizerFast.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")



In [7]:
import tqdm
device = 'cuda:0'
model.eval()
model.to(device)
dataset_translated = {}

for dataset_name, dataset_data in datasets.items():
    input_lang = "fr"
    output_lang = "en"
    if('squad' in dataset_name):
        input_lang = "en"
        output_lang = "fr"
    dataset_translated[dataset_name] = dataset_data

    tokenizer.src_lang = LANGUAGE_MAP[input_lang]
    for dataset_split_name, dataset_split_data in tqdm.tqdm(dataset_data.items()):
        for i, document in enumerate(dataset_split_data):
            for j, qa in enumerate(document['qas']):
                input_data = tokenizer([qa['question']], return_tensors='pt')
                output_data = model.generate(**{k: v.to(device) for k, v in input_data.items()}, 
                                             forced_bos_token_id=tokenizer.lang_code_to_id[LANGUAGE_MAP[output_lang]])
                output_text = tokenizer.batch_decode(output_data, skip_special_tokens=True)
                dataset_translated[dataset_name][dataset_split_name][i]['qas'][j]['question'] = output_text[0]
                break


  0%|                                                                                                             | 0/3 [00:00<?, ?it/s]/people/gerald/lib/miniconda3/envs/annotationbob/lib/python3.8/site-packages/transformers/generation_utils.py:1202: UserWarning: Neither `max_length` nor `max_new_tokens` have been set, `max_length` will default to 200 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:22<00:00, 22.94s/it]


In [12]:
for dataset_name, dataset_data in dataset_translated.items():
    input_lang = "fr"
    output_lang = "en"
    if('squad' in dataset_name):
        input_lang = "en"
        output_lang = "fr"
    for dataset_split_name, dataset_split_data in tqdm.tqdm(dataset_data.items()):
        dataset_folder = os.path.join(data_folder, 
                               dataset_name + "-" + 
                                      input_lang + "-" +
                                      output_lang)
        os.makedirs(dataset_folder, exist_ok=True)
        with open(os.path.join(dataset_folder, 
                               dataset_split_name), 'w') as f :
            json.dump(dataset_split_data, f)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 48.40it/s]
